数据来源平台：爬取印尼四大电商平台数据：lazada，shopee，tokopedia，bukalapak

爬取品类 ：按顺序依次为：鞋，表，包，饰品

价位区间：按200-300，及300以上两个价位区间爬取(饰品稍等，待观察各品台基础价位)

销量：爬出月销量1500以上的sku

https://www.tokopedia.com/
https://www.bukalapak.com/

## shopee
月销
## 奇怪 Get 不到 product name

In [7]:
from selenium import webdriver
import time,re,csv
from pyquery import PyQuery as pq
import requests
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import selenium.webdriver.support.expected_conditions as EC
import selenium.webdriver.support.ui as WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.keys import Keys

def get_list(class_name,class_url,save_file,monthly_sales):
    
    # Chrome object

    print('Getting class item list from Shopee ...')
    driver = webdriver.Chrome()  
    driver.maximize_window()
    driver.get(class_url) 
    time.sleep(2)

    for i in range(4):
        driver.find_element_by_xpath("/html/body").send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        

    items = driver.find_elements_by_css_selector('div.col-xs-2-4.shopee-search-item-result__item')
    item_number = len(items)
    if item_number == 50:
        with open(save_file,'a',encoding='utf-8') as w:    
            for i in items:
                #print(i.find_element_by_css_selector('._1NoI8_._2gr36I'))
                
                #pro_name = i.find_element_by_css_selector('div._1NoI8_._2gr36I').text
                pro_name = i.find_element_by_css_selector("[class^=_1NoI8_]").text
                price_info  = i.find_element_by_css_selector('div._36zw98').text
                
                price_range = re.sub(r'\.','',price_info)
    
                if '-' in price_range:
                    a = re.findall('Rp(.*) - Rp(.*)',price_range)
                    price = (int(a[0][0])+int(a[0][1]))/2
                else:
                    price = int(re.search('Rp(.*)',price_range).group(1))
                
                info = i.find_element_by_css_selector('div._2-i6yP').text
                monthly_sales = int(re.search('(\d+) Terjual',info).group(1))
                pro_url = i.find_element_by_tag_name('a').get_attribute('href')
                pic_info = i.find_element_by_css_selector('div._1T9dHf._3XaILN').get_attribute('style')
                pic_url = re.search(r'url\("(.*)"\);',pic_info).group(1)
                if monthly_sales >= sales_select:
                    print('class_name:{}\nproduct name:{}\nprice_info:{}\nprice:{}\nproduct url:{}\nmonthly sales:{}\npicture url:{}\n'.format(class_name,pro_name,price_info,price,pro_url,monthly_sales,pic_url))
                    #w.write('{},{}\n'.foramt(url,price))
                
    else:
        print('number of items is{}, waiting 10 seconds to reload'.foramt(item_number))
        #time.sleep(10)
        #get_list(class_url,save_file)

    
class_name = '靴子'
class_url = 'https://shopee.co.id/Boots-cat.35.1033?page=0&sortBy=sales'
save_file = r'class_item_list_over3000.csv'
sales_select = 1500

get_list(class_name,class_url,save_file,sales_select)

Getting class item list from Shopee ...
class_name:靴子
product name:
price_info:Rp198.000 - Rp279.000
price:238500.0
product url:https://shopee.co.id/Sepatu-Boots-Humm3r-Golem-size-39-44-dan-45-48-i.40377301.933066496
monthly sales:1577
picture url:https://cf.shopee.co.id/file/c11757c079d3611d2ff9e2beef2ef41a_tn



## pyquery

### 奇怪 get 不到 product url
`pro_url = i.find('div a').attr('href') `

#### 原因是打印出html代码，会发现包含  `xmlns="http://www.w3.org/1999/xhtml"`

```
<div xmlns="http://www.w3.org/1999/xhtml" class="col-xs-2-4 shopee-search-item-result__item"><div><a href="/Sepatu-Boots-Humm3r-Golem-size-39-44-dan-45-48-i.40377301.933066496"><div class="_1gkBDw"><div class="_3ZDC1p"><div class="_1T9dHf _3XaILN" style="background-image: url(&quot;https://cf.shopee.co.id/file/c11757c079d3611d2ff9e2beef2ef41a_tn&quot;)
```

但是pyquery中的选择器并没有错误，但是运行结果一直是None。这是为什么呢？后来通过查看相关文档得知，pyquery解析的是html类型的字符串，但是上面的类型是xhtml，所以会获取不到元素。可以在pq()方法初始化字符串时加上parser="html"告诉pyquery使用html规范解析，即可解决上述问题。
https://blog.csdn.net/qq_40176258/article/details/85041089

`pq(html,parser="html")`


再次打印出html代码，会发现已经不包含  `xmlns="http://www.w3.org/1999/xhtml"`

[官方文档](https://pythonhosted.org/pyquery/tips.html?highlight=xhtml)

In [2]:
from selenium import webdriver
import time,re,csv,pymongo,os
from pyquery import PyQuery as pq
import requests
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
import selenium.webdriver.support.expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.webdriver.common.keys import Keys


def get_list(save_file,category,class_name,class_url,sales_select,page):
    
    print('Getting class item list from Shopee ...') 
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(class_url.format(page)) 
    time.sleep(2)
    for i in range(4):
        driver.find_element_by_xpath("/html/body").send_keys(Keys.PAGE_DOWN)
        time.sleep(2)
    html = driver.page_source
    doc = pq(html,parser="html")
    item_list = doc('.col-xs-2-4.shopee-search-item-result__item')
    num = 1
    with open(save_file,'a',encoding='utf-8-sig', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['category','main class','item name','price info','price','monthly sales','item url','item picture'])
        
        for i in item_list.items():
            try:
                info = i.find('._2-i6yP').text()
                monthly_sales = int(re.search('(\d+) Terjual',info).group(1))    
                if monthly_sales >= sales_select:
                    num += 1
                    pro_name = i.find('._1NoI8_._2gr36I').text()
                    price_info  = i.find('._1w9jLI._37ge-4._2XtIUk').text()
                    price_range = re.sub(r'\.|\n','',price_info)
                    if '-' in price_range:
                        a = re.findall('Rp(.*) - Rp(.*)',price_range)
                        price = (int(a[0][0])+int(a[0][1]))/2
                    else:
                        price = int(re.search('Rp(.*)',price_range).group(1))
                    prefix = 'https://shopee.co.id'
                    pro_url = prefix+i.find('div a').attr('href') 
                    pic_info = i.find('._1T9dHf._3XaILN').attr('style')
                    print(type(pic_info))
                    pic_url = re.search(r'url\("(.*)_tn"\);',str(pic_info)).group(1).strip()
                    print('category:{}\nclass name:{}\nproduct name:{}\nprice info:{}\nprice:{}\nproduct url: {}\nmonthly sales:{}\npicture url: {}\n'.format(category,class_name,pro_name,price_range,price,pro_url,monthly_sales,pic_url))
                    csvfile.write('{},{},{},{},{},{},{},{}\n'.format(category,class_name,pro_name,price_range,price,monthly_sales,pro_url,pic_url)) 
                    pic_name = ' '.join(re.sub('[^a-z0-9A-Z_\-\s]','',pro_name,count=0).split())
                    save_image(pic_url,pic_name,category)

                    item = {
                        'category':category,
                        'main class':class_name,
                        'item name':pro_name,
                        'price info':price_range,
                        'price':price,
                        'monthly sales':monthly_sales,
                        'item url':pro_url,
                        'item picture':pic_url
                    }
                    save_to_mongo(item)
            except:
                print('something got worng, pass this item',pro_name)
                continue

    if num == len(item_list):
        page += 1
        get_list(save_file,category,class_name,class_url,sales_select,page)
                
        
def save_image(img_url,pic_name,category):
    print('{} saving picture... {}'.format(pic_name,img_url))
    img_response = requests.get(img_url,headers=headers)
    path = r'{}'.format(category)
    isExists = os.path.exists(path)
    if not isExists:
        os.makedirs(path)
    with open(r'{}\{}.png'.format(path,pic_name),'ab') as f:
        f.write(img_response.content)          
                        
def save_to_mongo(result):
    MONGO_URL = 'localhost'
    MONGO_DB = 'sales_crawl'
    MONGO_COLLECTION = '20190302'
    client = pymongo.MongoClient(MONGO_URL)
    db = client[MONGO_DB]
    collection = db[MONGO_COLLECTION]
    try:
        if collection.insert_one(result):
            print('save to mongodb')
    except Exception:
        print('failed to save to mongodb')

headers = {'User-Agent': 'User-Agent:Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'}            
category = 'MenShoes'
class_name = 'Boots'
class_url = 'https://shopee.co.id/Boots-cat.35.1033?page={}&sortBy=sales'
save_file = r'test_500.csv'
sales_select = 500
page = 0

get_list(save_file,category,class_name,class_url,sales_select,page)

Getting class item list from Shopee ...
<class 'str'>
category:MenShoes
class name:Boots
product name:Sepatu Boots Humm3r Golem size 39-44 dan 45-48
price info:Rp198000 - Rp279000
price:238500.0
product url: https://shopee.co.id/Sepatu-Boots-Humm3r-Golem-size-39-44-dan-45-48-i.40377301.933066496
monthly sales:1570
picture url: https://cf.shopee.co.id/file/c11757c079d3611d2ff9e2beef2ef41a

Sepatu Boots Humm3r Golem size 39-44 dan 45-48 saving picture... https://cf.shopee.co.id/file/c11757c079d3611d2ff9e2beef2ef41a
save to mongodb
<class 'str'>
category:MenShoes
class name:Boots
product name:Boots Dondhicero Millenium 39-44
price info:Rp219000 - Rp289000
price:254000.0
product url: https://shopee.co.id/Boots-Dondhicero-Millenium-39-44-i.40377301.947462866
monthly sales:1291
picture url: https://cf.shopee.co.id/file/21997f8f72825925a10e9fcf4c04b3fe

Boots Dondhicero Millenium 39-44 saving picture... https://cf.shopee.co.id/file/21997f8f72825925a10e9fcf4c04b3fe
save to mongodb
<class 'str'

save to mongodb
<class 'str'>
category:MenShoes
class name:Boots
product name:Sepatu Boots Humm3r Colombus Termurah
price info:Rp200000
price:200000
product url: https://shopee.co.id/Sepatu-Boots-Humm3r-Colombus-Termurah-i.28071026.1321055710
monthly sales:1019
picture url: https://cf.shopee.co.id/file/4d1b0e326083b098febea1ff2f47e035

Sepatu Boots Humm3r Colombus Termurah saving picture... https://cf.shopee.co.id/file/4d1b0e326083b098febea1ff2f47e035
save to mongodb
<class 'str'>
category:MenShoes
class name:Boots
product name:Sepatu Boots Humm3r Apollo 39-44
price info:Rp199000
price:199000
product url: https://shopee.co.id/Sepatu-Boots-Humm3r-Apollo-39-44-i.40377301.945034026
monthly sales:923
picture url: https://cf.shopee.co.id/file/d2e49eed868c7da8d7ef28dd6cca392c

Sepatu Boots Humm3r Apollo 39-44 saving picture... https://cf.shopee.co.id/file/d2e49eed868c7da8d7ef28dd6cca392c
save to mongodb
<class 'str'>
category:MenShoes
class name:Boots
product name:Sepatu Boots Humm3r Colombu

### 报错 TypeError: expected string or bytes-like object
（预定的数据类型或者字节对象相关） 一般为数据类型不匹配造成的

```
---> 49                 pic_url = re.search(r'url\("(.*)_tn"\);',pic_info).group(1).strip()
     50                 print('category:{}\nclass name:{}\nproduct name:{}\nprice info:{}\nprice:{}\nproduct url: {}\nmonthly sales:{}\npicture url: {}\n'.format(category,class_name,pro_name,price_range,price,pro_url,monthly_sales,pic_url))
     51                 csvfile.write('{},{},{},{},{},{},{},{}\n'.format(category,class_name,pro_name,price_range,price,monthly_sales,pro_url,pic_url))

c:\users\win7\appdata\local\programs\python\python36-32\lib\re.py in search(pattern, string, flags)
    180     """Scan through string looking for a match to the pattern, returning
    181     a match object, or None if no match was found."""
--> 182     return _compile(pattern, flags).search(string)
    183 
    184 def sub(pattern, repl, string, count=0, flags=0):

TypeError: expected string or bytes-like object
```
### 原因是 
`pic_info = i.find('._1T9dHf._3XaILN').attr('style')` 返回的是 `nonetype` 类型


用正则表达式处理数据时时出现错误,正则是用来匹配字符串类型，
因此在正则表达式之前，转换一下数据类型，就解决了问题
str()

[参考文档](https://blog.csdn.net/weixin_42105977/article/details/80390957)